# Original Dataset - ML Test

In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import sqlalchemy
from sqlalchemy import create_engine, inspect

import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from pprint import pprint

import os
import time
from datetime import datetime

%run functions.ipynb

In [2]:
# Time the run
start_time = time.time()

In [3]:
# Check for untitled_project folder
if os.path.exists("untitled_project"):
    input("DELETE untitled_project")

## Import datasets

In [4]:
# Import the data
engine = create_engine("sqlite:///voice.sqlite")

# View all of the classes
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

['aval',
 'bval',
 'demographic',
 'diagnosis',
 'gval',
 'habits',
 'rval',
 'spectrogram']

In [5]:
# Initialise a dictionary to hold dataframes
dataframes = dict()

# Loop through each table
for table in table_names:
    
    # Dataframe name
    df_name = f'{table}_df'
    
    # Create dataframe
    dataframes[df_name] = pd.read_sql(
        f'SELECT * FROM {table}',
        engine
    )

In [6]:
# Merge dataframes
merged_df = pd.merge(
    dataframes['demographic_df'],
    dataframes['diagnosis_df'],
    how = 'inner',
    on = 'id'
)

merged_df = pd.merge(
    merged_df,
    dataframes['habits_df'],
    how = 'inner',
    on = 'id'
)

# Display merged_df
merged_df.head()

,id,age,gender,occupation_status,diagnosis,subtype,vhi_score,rsi_score,alcohol_consumption,alcohol_pd,...,chocolate,chocolate_grams_pd,coffee,coffee_pd,citrus_fruits,citrus_fruits_pd,soft_cheese,soft_cheese_pd,tomatoes,water_litres_pd
0,voice100,24,m,unknown,healthy,no subtype,0,5,casual,0.36,...,sometimes,30,always,3,never,0.00,almost always,100,never,1.5
1,voice101,60,m,unknown,healthy,no subtype,80,10,nondrinker,0.00,...,sometimes,30,always,4,never,0.00,sometimes,100,sometimes,1.5
2,voice192,22,m,cook,hyperkinetic dysphonia,no subtype,0,10,nondrinker,0.00,...,always,14,always,3,almost always,1.17,sometimes,100,sometimes,2.5
3,voice193,46,f,housewife,hyperkinetic dysphonia,no subtype,0,36,casual,0.36,...,sometimes,30,always,2,sometimes,1.00,sometimes,100,sometimes,1.0
4,voice008,51,f,researcher,reflux laryngitis,no subtype,19,15,casual,0.36,...,almost always,20,always,2,almost always,1.00,sometimes,100,almost always,1.0


## Preprocessing

### Separate the target and feature variables

In [7]:
# Drop the 'id' column
no_id_df = merged_df.drop(columns=['id'])

In [8]:
# Define the target variables
target_var = ['diagnosis', 'subtype']
y = no_id_df[target_var]

# Define the feature variables
X = no_id_df.drop(columns=target_var)

### Binary classification - `diagnosis`

In [9]:
# Encode the target variable, ignore subtype
y = y['diagnosis'].apply(encode_binary)
y

0      0
1      0
2      1
3      1
4      1
      ..
199    0
200    1
201    1
202    0
203    0
Name: diagnosis, Length: 204, dtype: int64

### Bin `occupation_status` column

In [10]:
# Use limit_unique() function to bin the column
limit_unique(X, 10, ['occupation_status'])

occupation_status
unknown               41
researcher            41
other                 25
employee              25
housewife             23
student               16
technical operator    12
singer                10
pensioner              6
doctor                 5
Name: count, dtype: int64
Number of unique values: 10



### Encode feature columns

#### Encoding
- `smoker` column
	- `0` for `no`
	- `1` for `casual`
	- `2` for `yes`
- `alcohol_consumption` column
	- `0` for `nondrinker`
	- `1` for `casual`
	- `2` for `habitual`
- `carbonated_beverages`, `tomatoes`, `coffee`, `chocolate`, `soft_cheese`, `citrus_fruits` columns
	- `0` for `never`
	- `1` for `almost never`
	- `2` for `sometimes`
	- `3` for `almost always`
	- `4` for `always`

In [11]:
# Define the maps
smoker_map = {
    'no': 0,
    'casual': 1,
    'yes': 2
}

alcohol_map = {
    'nondrinker': 0,
    'casual': 1,
    'habitual': 2
}

habit_map = {
    'never': 0,
    'almost never': 1,
    'sometimes': 2,
    'almost always': 3,
    'always': 4
}

In [12]:
# Apply the label encoding using the maps
X['smoker'] = X['smoker'].map(smoker_map)
X['alcohol_consumption'] = X['alcohol_consumption'].map(alcohol_map)

# Habit columns
habit_cols = [
    'carbonated_beverages', 'tomatoes',
    'coffee', 'chocolate',
    'soft_cheese', 'citrus_fruits'
]

# Use a loop for the habit columns
for habit in habit_cols:
    X[habit] = X[habit].map(habit_map)

In [13]:
# Remove the occupation_status column
X = X.drop(columns=['occupation_status'])

In [14]:
# Encode the categorical columns using get_dummies
# categorical_hot = ['gender', 'occupation_status']
categorical_hot = ['gender']

# One-hot encoding
encoded_columns = pd.get_dummies(X[categorical_hot]).astype(int)

# Update the feature dataframe
X.drop(categorical_hot, axis=1, inplace=True)
X = pd.concat([X, encoded_columns], axis=1)

In [15]:
# Display the dataframe
X.info()
X.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   age                   204 non-null    int64  
 1   vhi_score             204 non-null    int64  
 2   rsi_score             204 non-null    int64  
 3   alcohol_consumption   204 non-null    int64  
 4   alcohol_pd            204 non-null    float64
 5   smoker                204 non-null    int64  
 6   cigarettes_pd         204 non-null    int64  
 7   carbonated_beverages  204 non-null    int64  
 8   carbonated_pd         204 non-null    float64
 9   chocolate             204 non-null    int64  
 10  chocolate_grams_pd    204 non-null    int64  
 11  coffee                204 non-null    int64  
 12  coffee_pd             204 non-null    int64  
 13  citrus_fruits         204 non-null    int64  
 14  citrus_fruits_pd      204 non-null    float64
 15  soft_cheese           2

,age,vhi_score,rsi_score,alcohol_consumption,alcohol_pd,smoker,cigarettes_pd,carbonated_beverages,carbonated_pd,chocolate,...,coffee,coffee_pd,citrus_fruits,citrus_fruits_pd,soft_cheese,soft_cheese_pd,tomatoes,water_litres_pd,gender_f,gender_m
0,24,0,5,1,0.36,0,0,3,3.00,2,...,4,3,0,0.00,3,100,0,1.5,0,1
1,60,80,10,0,0.00,0,0,3,3.00,2,...,4,4,0,0.00,2,100,2,1.5,0,1
2,22,0,10,0,0.00,0,0,0,0.00,4,...,4,3,3,1.17,2,100,2,2.5,0,1
3,46,0,36,1,0.36,2,15,2,0.61,2,...,4,2,2,1.00,2,100,2,1.0,1,0
4,51,19,15,1,0.36,0,0,1,0.09,3,...,4,2,3,1.00,2,100,3,1.0,1,0


### Split and Scale

In [16]:
# Split the preprocessed data to training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [17]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Hyperparameter Tuning

In [18]:
# Define the model parameters
number_input_features = len(X_train.columns)

# Maximum hidden layers (min. 2 for DL)
max_hidden_layers = 3

# Maximum neurons per hidden layer
max_num_neurons = number_input_features * 2 - 1

# Step count
step_count = 5

# Hidden layer activation functions
activation_functions = [
    'relu', 'leaky_relu', 'tanh',
    'elu', 'selu', 'exponential',
    'softmax', 'softplus'
]

# Number of max epochs
tuner_max_epochs = 20
search_max_epochs = 20

# Hyperband iterations
hp_iterations = 2

In [19]:
# Initialise the Hyperband tuner
tuner = kt.Hyperband(
    create_model,
    objective = "val_accuracy",
    max_epochs = tuner_max_epochs,
    hyperband_iterations = hp_iterations
)

In [20]:
# Find the best hyperparameters
tuner.search(
    X_train_scaled,
    y_train,
    epochs = search_max_epochs,
    validation_data = (X_test_scaled, y_test)
)

Trial 60 Complete [00h 00m 01s]
val_accuracy: 0.7254902124404907

Best val_accuracy So Far: 0.8039215803146362
Total elapsed time: 00h 00m 28s


## Compile, Train, Evaluate the Best Model

In [21]:
# Get the top 3 model hyperparameters
top3_hyper = tuner.get_best_hyperparameters(3)

for model in top3_hyper:
    pprint(model.values)

{'activation_layer_0': 'relu',
 'activation_layer_1': 'tanh',
 'activation_layer_2': 'leaky_relu',
 'num_layers': 1,
 'tuner/bracket': 1,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 1,
 'tuner/trial_id': '0020',
 'units_layer_0': 21,
 'units_layer_1': 31,
 'units_layer_2': 6}
{'activation_layer_0': 'leaky_relu',
 'activation_layer_1': 'softplus',
 'activation_layer_2': 'softplus',
 'num_layers': 1,
 'tuner/bracket': 2,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 2,
 'tuner/trial_id': '0043',
 'units_layer_0': 36,
 'units_layer_1': 31,
 'units_layer_2': 26}
{'activation_layer_0': 'relu',
 'activation_layer_1': 'relu',
 'activation_layer_2': 'leaky_relu',
 'num_layers': 2,
 'tuner/bracket': 1,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 1,
 'tuner/trial_id': '0019',
 'units_layer_0': 16,
 'units_layer_1': 26,
 'units_layer_2': 21}


In [22]:
# Get the top model
best_hyper = top3_hyper[0]
best_hyper.values

{'activation_layer_0': 'relu',
 'units_layer_0': 21,
 'num_layers': 1,
 'units_layer_1': 31,
 'activation_layer_1': 'tanh',
 'units_layer_2': 6,
 'activation_layer_2': 'leaky_relu',
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0020'}

In [23]:
# Define the model parameters
number_input_features = len(X_train.columns)
hidden_layer0_neurons = best_hyper.values['units_layer_0']
hidden_layer0_activation = best_hyper.values['activation_layer_0']

# Return the number of hidden layers
total_hidden = best_hyper.values['num_layers'] + 1

# Define the output layer
output_layer_neurons = 1
output_layer_activation = "sigmoid"

# Model compilation
compile_loss = "binary_crossentropy"
compile_opt = "adam"

# Train model
model_train_epochs = 100

In [24]:
# Initialise the sequential model
nn = tf.keras.models.Sequential()

# Create the input layer and first hidden layer
nn.add(tf.keras.layers.Dense(
    units = hidden_layer0_neurons,
    activation = hidden_layer0_activation,
    input_dim = number_input_features
))
        
# Create additional hidden layers
for layer in range(1, total_hidden):
    nn.add(tf.keras.layers.Dense(
        units = best_hyper.values[f'units_layer_{layer}'],
        activation = best_hyper.values[f'activation_layer_{layer}']
    ))

# Create the output layer
nn.add(tf.keras.layers.Dense(
    units = output_layer_neurons,
    activation = output_layer_activation
))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 21)                462       
                                                                 
 dense_4 (Dense)             (None, 31)                682       
                                                                 
 dense_5 (Dense)             (None, 1)                 32        
                                                                 
Total params: 1176 (4.59 KB)
Trainable params: 1176 (4.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
# Compile the model
nn.compile(
    loss = compile_loss,
    optimizer = compile_opt,
    metrics = ["accuracy"]
)

In [26]:
# Train the model
fit_model = nn.fit(
    X_train_scaled,
    y_train,
    epochs = model_train_epochs,
    verbose = 1
)

Epoch 1/100
5/5 [==============================] - 0s 922us/step - loss: 0.8439 - accuracy: 0.3856
Epoch 2/100
5/5 [==============================] - 0s 747us/step - loss: 0.7529 - accuracy: 0.4837
Epoch 3/100
5/5 [==============================] - 0s 859us/step - loss: 0.6872 - accuracy: 0.5425
Epoch 4/100
5/5 [==============================] - 0s 840us/step - loss: 0.6360 - accuracy: 0.6209
Epoch 5/100
5/5 [==============================] - 0s 818us/step - loss: 0.6105 - accuracy: 0.6601
Epoch 6/100
5/5 [==============================] - 0s 746us/step - loss: 0.5877 - accuracy: 0.7124
Epoch 7/100
5/5 [==============================] - 0s 985us/step - loss: 0.5707 - accuracy: 0.7255
Epoch 8/100
5/5 [==============================] - 0s 791us/step - loss: 0.5596 - accuracy: 0.7386
Epoch 9/100
5/5 [==============================] - 0s 947us/step - loss: 0.5490 - accuracy: 0.7516
Epoch 10/100
5/5 [==============================] - 0s 805us/step - loss: 0.5400 - accuracy: 0.7516
Epoch 11/

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(
    X_test_scaled,
    y_test,
    verbose = 2
)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 0.5281 - accuracy: 0.7843 - 37ms/epoch - 19ms/step
Loss: 0.5280581712722778, Accuracy: 0.7843137383460999


In [28]:
current_time = datetime.now()
current_time.second

4

In [29]:
# Create dictionary to save the results
results_dict = dict()

results_dict['timestamp'] = current_time
results_dict['runtime'] = time.time() - start_time
results_dict['model_loss'] = model_loss
results_dict['model_accuracy'] = model_accuracy

In [30]:
# Print the model architecture
print(f"Input features: {number_input_features}")
print(f"Hidden Layer 0: {hidden_layer0_activation}, {hidden_layer0_neurons}")

# Populate results_dict with model details
results_dict['input_features'] = number_input_features
results_dict['hlayer_0_activation'] = hidden_layer0_activation
results_dict['hlayer_0_neurons'] = hidden_layer0_neurons

for layer in range(1, total_hidden):
    activation = best_hyper.values[f'activation_layer_{layer}']
    neurons = best_hyper.values[f'units_layer_{layer}']
    print(f"Hidden Layer {layer}: {activation}, {neurons}")
    
    results_dict[f'hlayer_{layer}_activation'] = best_hyper.values[f'activation_layer_{layer}']
    results_dict[f'hlayer_{layer}_neurons'] = best_hyper.values[f'units_layer_{layer}']

results_dict['olayer_neurons'] = output_layer_neurons
results_dict['olayer_activation'] = output_layer_activation

# Populate results_dict with tuning details
results_dict['tuning_max_hidden'] = max_hidden_layers
results_dict['tuning_max_neurons'] = max_num_neurons
results_dict['tuning_step_count'] = step_count
results_dict['activation_functions'] = str(activation_functions)
results_dict['tuning_tuner_epochs'] = tuner_max_epochs
results_dict['tuning_search_epochs'] = search_max_epochs
results_dict['tuning_hp_iterations'] = hp_iterations

# Populate results_dict with compilation details
results_dict['compile_loss'] = compile_loss
results_dict['compile_optimizer'] = compile_opt

Input features: 21
Hidden Layer 0: relu, 21
Hidden Layer 1: tanh, 31


In [31]:
# Change message
change_message = input("Changes from previous iteration: ")

# Append to results_dict
results_dict['change_message'] = change_message

Changes from previous iteration:  dropped occupation_status column


In [32]:
# Display the dictionary
results_dict

{'timestamp': datetime.datetime(2024, 1, 2, 17, 49, 4, 691325),
 'runtime': 34.3794150352478,
 'model_loss': 0.5280581712722778,
 'model_accuracy': 0.7843137383460999,
 'input_features': 21,
 'hlayer_0_activation': 'relu',
 'hlayer_0_neurons': 21,
 'hlayer_1_activation': 'tanh',
 'hlayer_1_neurons': 31,
 'olayer_neurons': 1,
 'olayer_activation': 'sigmoid',
 'tuning_max_hidden': 3,
 'tuning_max_neurons': 41,
 'tuning_step_count': 5,
 'activation_functions': "['relu', 'leaky_relu', 'tanh', 'elu', 'selu', 'exponential', 'softmax', 'softplus']",
 'tuning_tuner_epochs': 20,
 'tuning_search_epochs': 20,
 'tuning_hp_iterations': 2,
 'compile_loss': 'binary_crossentropy',
 'compile_optimizer': 'adam',
 'change_message': 'dropped occupation_status column'}

In [33]:
# Convert the dictionary to a dataframe
results_df = pd.DataFrame(results_dict, index=[0])
results_df.head()

,timestamp,runtime,model_loss,model_accuracy,input_features,hlayer_0_activation,hlayer_0_neurons,hlayer_1_activation,hlayer_1_neurons,olayer_neurons,...,tuning_max_hidden,tuning_max_neurons,tuning_step_count,activation_functions,tuning_tuner_epochs,tuning_search_epochs,tuning_hp_iterations,compile_loss,compile_optimizer,change_message
0,2024-01-02 17:49:04.691325,34.379415,0.528058,0.784314,21,relu,21,tanh,31,1,...,3,41,5,"['relu', 'leaky_relu', 'tanh', 'elu', 'selu', ...",20,20,2,binary_crossentropy,adam,dropped occupation_status column


In [34]:
# Performance tracker
tracker_path = "../resources/performance_tracker.csv"

# Model percentage
model_pct = round(model_accuracy, 3)

# Check if the CSV exists
if os.path.exists(tracker_path):
    
    # Read the existing CSV
    tracker_df = pd.read_csv(tracker_path)
    
    # Append the new row of data
    updated_df = pd.concat([tracker_df, results_df], ignore_index=True)
    
    # Update the CSV file
    updated_df.to_csv(tracker_path, index=False)
    
    # Export model to HDF5 file
    nn.save(f'../models/run_{len(tracker_df)}_{model_pct}.h5', save_format='h5')
        
else:    
    # Export to CSV
    results_df.to_csv(tracker_path, index=False)
    
    # Export model to HDF5 file
    nn.save(f'../models/run_0_{model_pct}.h5', save_format='h5')

/Users/alyssahondrade/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
